In [1]:
import pymongo
import pandas as pd

In [2]:
client = pymongo.MongoClient("localhost", 27017)

In [3]:
# Creating a database
db = client['twitter-covid2']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'twitter-covid2')

In [4]:
# Create a collection
collection = db["tweets"]

In [5]:
covid2_df = pd.read_json('corona-out-2',lines = True)

In [6]:
covid2_df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,favorited,retweeted,filter_level,lang,timestamp_ms,display_text_range,extended_tweet,possibly_sensitive,extended_entities,withheld_in_countries
0,2020-04-12 18:27:25+00:00,1249403767180668930,1249403767180668928,RT @nuffsaidny: wishing death on people is wei...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,False,False,low,en,2020-04-12 18:27:25.552,NaN,NaN,NaN,NaN,NaN
1,2020-04-12 18:27:25+00:00,1249403768023678982,1249403768023678976,"RT @lale_karanfil: In Turkey, there are 300 th...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,False,False,low,en,2020-04-12 18:27:25.753,NaN,NaN,NaN,NaN,NaN
2,2020-04-12 18:27:26+00:00,1249403769193779202,1249403769193779200,RT @umesh_agr: Thank You Sir !!\nReally thankf...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,False,False,low,en,2020-04-12 18:27:26.032,NaN,NaN,NaN,NaN,NaN
3,2020-04-12 18:27:26+00:00,1249403769567227906,1249403769567227904,RT @meysimek: Turkey is so stubborn to change ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,False,False,low,en,2020-04-12 18:27:26.121,NaN,NaN,NaN,NaN,NaN
4,2020-04-12 18:27:26+00:00,1249403770435493888,1249403770435493888,RT @biannagolodryga: This image.\nThis quote.\...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,False,False,low,en,2020-04-12 18:27:26.328,NaN,NaN,NaN,NaN,NaN


In [7]:
user_df = pd.json_normalize(covid2_df['user'])

In [11]:
# Adding columns from user to tweets data

covid2_df['user_id_str']=user_df['id_str']

In [12]:
covid2_df['user_verified']=user_df['verified']
covid2_df['user_screen_name']=user_df['screen_name']

In [13]:
covid2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18518 entries, 0 to 18517
Data columns (total 40 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 18518 non-null  datetime64[ns, UTC]
 1   id                         18518 non-null  int64              
 2   id_str                     18518 non-null  int64              
 3   text                       18518 non-null  object             
 4   source                     18518 non-null  object             
 5   truncated                  18518 non-null  bool               
 6   in_reply_to_status_id      2528 non-null   float64            
 7   in_reply_to_status_id_str  2528 non-null   float64            
 8   in_reply_to_user_id        2734 non-null   float64            
 9   in_reply_to_user_id_str    2734 non-null   float64            
 10  in_reply_to_screen_name    2734 non-null   object             
 11  us

In [14]:
# Iterate through the rows of the DataFrame and insert each row as a separate document in the collection
for index, row in covid2_df.iterrows():
    tweet = {
        'created_at': row['created_at'],
        'id': row['id'],
        'id_str': row['id_str'],
        'text': row['text'],
        'source': row['source'],
        'truncated': row['truncated'],
        'in_reply_to_status_id': row['in_reply_to_status_id'],
        'in_reply_to_status_id_str': row['in_reply_to_status_id_str'],
        'in_reply_to_user_id': row['in_reply_to_user_id'],
        'in_reply_to_user_id_str': row['in_reply_to_user_id_str'],
        'in_reply_to_screen_name': row['in_reply_to_screen_name'],
        'geo': row['geo'],
        'coordinates': row['coordinates'],
        'place': row['place'],
        'contributors': row['contributors'],
        'retweeted_status': row['retweeted_status'],
        'quoted_status_id': row['quoted_status_id'],
        'quoted_status_id_str': row['quoted_status_id_str'],
        'quoted_status': row['quoted_status'],
        'quoted_status_permalink': row['quoted_status_permalink'],
        'is_quote_status': row['is_quote_status'],
        'quote_count': row['quote_count'],
        'reply_count': row['reply_count'],
        'retweet_count': row['retweet_count'],
        'favorite_count': row['favorite_count'],
        'entities': row['entities'],
        'favorited': row['favorited'],
        'retweeted': row['retweeted'],
        'filter_level': row['filter_level'],
        'lang': row['lang'],
        'timestamp_ms': row['timestamp_ms'],
        'display_text_range': row['display_text_range'],
        'extended_tweet': row['extended_tweet'],
        'possibly_sensitive': row['possibly_sensitive'],
        'extended_entities': row['extended_entities'],
        'withheld_in_countries': row['withheld_in_countries'],
        'user_id_str': row['user_id_str'],
        'user_verified': row['user_verified'],
        'user_screen_name': row['user_screen_name']
    }
    collection.insert_one(tweet)
